In [18]:
import json
import re
import unicodedata
import pandas as pd
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from stop_words import get_stop_words

In [3]:
#CARGA DE DATOS
data_dict = {}
with open('C:/Users/adril/Downloads/PracticaNLP/Video_Games_5.json', "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        data_dict[idx] = json.loads(line)

In [6]:
#CREACIÓN DATAFRAME CON REVIEWS Y SENTIMIENTOS
reviews_text = []
reviews_sentiment = []

for review in data_dict.values():
    reviews_text.append(review.get('reviewText'))
    reviews_sentiment.append(review.get('overall'))

df_reviews = pd.DataFrame({
    'review': reviews_text,
    'sentiment': reviews_sentiment
})
df_reviews.dropna(subset=['review', 'sentiment'], inplace=True)
df_reviews.reset_index(drop=True, inplace=True)

In [12]:
# FUNCIÓN NORMALIZACIÓN
def sentence_normalization(sentence):
    sentence = unicodedata.normalize('NFKD', sentence).lower().encode('ascii', errors='ignore').decode('utf-8')
    sentence = re.sub(' +', ' ', ' '.join([word if word.isalpha() else '' for word in sentence.split()])).strip()
    return sentence

# FUNCIÓN STOPWORDS
def remove_stopwords(sentence, sw_list):
    sentence = ' '.join([word for word in sentence.split() if word not in sw_list])
    return sentence

# FUNCIÓN STEMMING
stemmer = PorterStemmer()
def apply_stemming(sentence): 
    return ' '.join([stemmer.stem(word) for word in sentence.split()])

sw_list = get_stop_words('en')

# FUNCIÓN FINAL DONDE SE APLICAN LOS ANTERIORES
def process_reviews(reviews, sw_list):
    processed_sentences = []
    for sent in reviews:
        if not sent != sent:
            sent = sentence_normalization(sent)
            sent = remove_stopwords(sent, sw_list)
            sent = apply_stemming(sent)
            processed_sentences.append(sent)
        else:
            processed_sentences.append('None')
    return processed_sentences

In [19]:
processed_reviews = process_reviews(df_reviews['review'], sw_list) # APLICACIÓN DE LA FUNCIÓN FINAL A MI COLUMNA DE REVIEWS
df_reviews['processedReview'] = processed_reviews # CREO NUEVA COLUMNA
df_reviews['processedReview'] = df_reviews['processedReview'].replace('', np.nan) # REEMPLAZO VACÍO POR NAN
df_reviews = df_reviews.dropna(subset=['processedReview']) # ELIMINO FILAS NAN